In [3]:
pip install pandas requests lxml


  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-macosx_10_13_universal2.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.4 MB/s  0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.4-cp312-cp312-macosx_10_13_universal2.whl (208 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
Using cached urllib3-2.6.3-py3-none-any.whl (131 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 54.8 MB/s  0:00:00
Using cached certifi-2026.1.4-py3-none-any.whl (152 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [pandas]2m6/7 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

url = "https://f1.fandom.com/wiki/Penalty_Points"

# 페이지 내 모든 테이블 읽기
tables = pd.read_html(url)

len(tables)


5

In [5]:
penalty_table = None

for t in tables:
    cols = [str(c).lower() for c in t.columns]
    if ("driver" in cols) and ("points" in cols):
        penalty_table = t
        break

penalty_table.head()


,Pos,No.,Driver,Points,Event,Date,Points.1,Reason
0,1,20,Kevin Magnussen,12 (now 0),NaN,NaN,NaN,NaN
1,1,20,Kevin Magnussen,12 (now 0),NaN,NaN,NaN,NaN
2,2,14,Fernando Alonso,8,2024 Australian GP,3/24/24,3,Dangerous driving at turn 6-7
3,2,14,Fernando Alonso,8,2024 Chinese GP,4/20/24,3,Causing a collision with Carlos Sainz in the S...
4,3,14,Fernando Alonso,8,2024 Austrian GP,6/29/24,2,Causing a collision with Zhou Guanyu in the sp...


In [7]:
df = penalty_table.copy()
df.columns = [c.strip() for c in df.columns]

df = df[["Driver", "Points.1", "Reason"]]
df


,Driver,Points.1,Reason
0,Kevin Magnussen,NaN,NaN
1,Kevin Magnussen,NaN,NaN
2,Fernando Alonso,3,Dangerous driving at turn 6-7
3,Fernando Alonso,3,Causing a collision with Carlos Sainz in the S...
4,Fernando Alonso,2,Causing a collision with Zhou Guanyu in the sp...
5,Max Verstappen,2,Collision with Lando Norris
6,Max Verstappen,2,Forcing Lando Norris off the track
7,Max Verstappen,1,VSC infringement
8,Max Verstappen,1,Impeding George Russell
9,Lance Stroll,3,Causing a collision with Daniel Ricciardo


In [8]:
df["Points.1"] = pd.to_numeric(df["Points.1"], errors="coerce")
df = df.dropna(subset=["Points.1"])


In [11]:
import pandas as pd
import numpy as np

def get_season_penalty_table(year: int) -> pd.DataFrame:
    url = f"https://f1.fandom.com/wiki/{year}_Formula_One_Season"
    tables = pd.read_html(url)

    target = None
    for t in tables:
        cols = [str(c).lower() for c in t.columns]
        if ("driver" in cols) and ("points" in cols):
            target = t.copy()
            break

    if target is None:
        return pd.DataFrame(columns=["Year", "Driver", "Points"])

    # 컬럼 정리
    target.columns = [str(c).strip() for c in target.columns]

    # 최소 컬럼
    target = target[["Driver", "Points"]].copy()
    target["Year"] = year

    # 1️⃣ Driver가 실제 이름이 아닌 행 제거 (Source, NaN 등)
    target["Driver"] = target["Driver"].astype(str)
    target = target[~target["Driver"].str.contains("source", case=False, na=False)]
    target = target[target["Driver"].str.len() > 2]

    # 2️⃣ Points에서 숫자만 추출
    target["Points"] = (
        target["Points"]
        .astype(str)
        .str.extract(r"(\d+)")
    )

    target["Points"] = pd.to_numeric(target["Points"], errors="coerce")

    # 3️⃣ Points가 없는 행 제거
    target = target.dropna(subset=["Points"])
    target["Points"] = target["Points"].astype(int)

    # 4️⃣ 연도 컬럼 확실히 숫자로 고정
    target["Year"] = int(year)

    return target[["Year", "Driver", "Points"]]


# 여러 시즌 수집
all_years = []
for y in range(2018, 2025):
    all_years.append(get_season_penalty_table(y))

season_points = pd.concat(all_years, ignore_index=True)

# 🔒 안전망: 연도 범위 재확인
season_points = season_points[season_points["Year"].between(2018, 2024)]

season_points.reset_index(drop=True, inplace=True)

# CSV 저장
season_points.to_csv(
    "penalty_points.csv",
    index=False,
    encoding="utf-8-sig"
)

season_points.head()


,Year,Driver,Points
0,2018,Romain Grosjean,7
1,2018,Lance Stroll,7
2,2018,Brendon Hartley,6
3,2018,Max Verstappen,6
4,2018,Sergey Sirotkin,6


In [13]:
import pandas as pd
import numpy as np

# Paths
input_path = "data/penalty_skill_by_season.csv"
output_path = "data/penalty_skill_by_season_scaled.csv"

# Hyperparameter: shrinkage to reduce small-sample distortion
K_LAPS = 500  # you can try 300~800 depending on how strong you want the smoothing

df = pd.read_csv(input_path)

# Safety: ensure numeric
df["Points"] = pd.to_numeric(df["Points"], errors="coerce").fillna(0)
df["TotalLaps"] = pd.to_numeric(df["TotalLaps"], errors="coerce")

# Guard: drop rows with missing or zero laps (cannot compute)
df = df.dropna(subset=["TotalLaps"])
df = df[df["TotalLaps"] > 0].copy()

# 1) Adjusted PenaltySkill to reduce extreme penalty for low TotalLaps
df["PenaltySkill_adj"] = -np.log1p(df["Points"] / (df["TotalLaps"] + K_LAPS))

# 2) Season-wise 0-10 scaling based on adjusted skill
def scale_0_10(x: pd.Series) -> pd.Series:
    x_min = x.min()
    x_max = x.max()
    if x_max > x_min:
        return 10 * (x - x_min) / (x_max - x_min)
    return pd.Series([10.0] * len(x), index=x.index)

df["PenaltySkill_0_10"] = df.groupby("Year")["PenaltySkill_adj"].transform(scale_0_10)

# Optional: round for readability (keep raw too if you want)
df["PenaltySkill_0_10"] = df["PenaltySkill_0_10"].round(6)

# Save
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"Saved: {output_path}")
print(df.head(10))


Saved: data/penalty_skill_by_season_scaled.csv
   Year            Driver  Points  TotalLaps  PenaltySkill  PenaltySkill_adj  \
0  2024    Oliver Bearman       4        170     -0.023257         -0.005952   
1  2024    Max Verstappen       8       1389     -0.005743         -0.004226   
2  2024       Liam Lawson       2        358     -0.005571         -0.002328   
3  2024  Franco Colapinto       2        398     -0.005013         -0.002225   
4  2024     Oscar Piastri       4       1443     -0.002768         -0.002057   
5  2024      Esteban Ocon       3       1238     -0.002420         -0.001725   
6  2024      Lance Stroll       3       1265     -0.002369         -0.001698   
7  2024      Lando Norris       3       1436     -0.002087         -0.001548   
8  2024   Alexander Albon       2       1141     -0.001751         -0.001218   
9  2024      Carlos Sainz       2       1345     -0.001486         -0.001083   

   PenaltySkill_0_10  
0           0.000000  
1           2.900169  
2  